In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle

In [2]:
 
with open("graphs_data", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [52]:
data_list = []
count = 0
for data in read_graph_data:
    if data.x.shape[0] == 0:
        continue
        
    if count < 180 :
        count +=1
        data_list.append(data)
    
    elif data.y == 1:
        data_list.append(data)
        
    

In [56]:
total = 0
for data in data_list:
    total+=data.y
    print(data.x.shape)
print(total)

torch.Size([12995, 1])
torch.Size([8409, 1])
torch.Size([10330, 1])
torch.Size([12756, 1])
torch.Size([12493, 1])
torch.Size([26679, 1])
torch.Size([25138, 1])
torch.Size([64941, 1])
torch.Size([72605, 1])
torch.Size([52394, 1])
torch.Size([39613, 1])
torch.Size([39254, 1])
torch.Size([49259, 1])
torch.Size([70124, 1])
torch.Size([66424, 1])
torch.Size([137604, 1])
torch.Size([137158, 1])
torch.Size([153259, 1])
torch.Size([250038, 1])
torch.Size([221339, 1])
torch.Size([192915, 1])
torch.Size([94516, 1])
torch.Size([5288, 1])
torch.Size([4123, 1])
torch.Size([4918, 1])
torch.Size([6450, 1])
torch.Size([6766, 1])
torch.Size([20566, 1])
torch.Size([30039, 1])
torch.Size([74271, 1])
torch.Size([73291, 1])
torch.Size([67166, 1])
torch.Size([48794, 1])
torch.Size([45520, 1])
torch.Size([60850, 1])
torch.Size([85819, 1])
torch.Size([82082, 1])
torch.Size([155868, 1])
torch.Size([175016, 1])
torch.Size([184059, 1])
torch.Size([280255, 1])
torch.Size([204999, 1])
torch.Size([182872, 1])
torch

In [47]:
data.x.shape[0]

12995

In [57]:
train_loader = DataLoader(data_list, batch_size=1,shuffle=True)
test_loader = DataLoader(data_list, batch_size=1,shuffle=False)

In [58]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = CGConv((mydataset.num_node_features, hidden_channels),mydataset.num_edge_features,batch_norm=True)
        self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [62]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# weights = torch.tensor([.05,.95]).to(device)
criterion = torch.nn.CrossEntropyLoss()

In [63]:
def train(accum_iter):
    model.train()
    for batch_idx,data in enumerate(train_loader):

        if data.x.shape[0] == 0:
            continue

        data = data.to(device)
        out = model(data.x, data.edge_index,data.batch)

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        
        if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
            optimizer.step()
            optimizer.zero_grad()
#             print(batch_idx)
        

    return loss.cpu().item()
    
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        
        if data.x.shape[0] == 0:
            continue

        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

# test(test_loader)

In [65]:

for epoch in range(100):
    
    loss = train(10)
    train_acc = test(test_loader) 
    print(f'{loss} {train_acc}')


0.671462893486023 0.6497175141242938
0.9581215381622314 0.5677966101694916
0.87684565782547 0.7033898305084746
0.42687878012657166 0.6101694915254238
0.9462871551513672 0.7005649717514124
0.7186324000358582 0.6610169491525424
1.3250409364700317 0.6864406779661016
0.4775136411190033 0.5621468926553672
0.2515844702720642 0.6214689265536724
0.23750272393226624 0.7146892655367232
0.0816783532500267 0.7062146892655368
0.32122576236724854 0.711864406779661
1.6321287155151367 0.7231638418079096
1.320317268371582 0.711864406779661
0.09555097669363022 0.7288135593220338
0.7050990462303162 0.7259887005649718
1.1936209201812744 0.6836158192090396
0.8472549915313721 0.7146892655367232
0.0902116522192955 0.7033898305084746
0.3086977005004883 0.6694915254237288
0.8970960378646851 0.7231638418079096
0.9651280641555786 0.6949152542372882
0.005530651658773422 0.6892655367231638
0.26009926199913025 0.7005649717514124
1.0181043148040771 0.7175141242937854
0.12711702287197113 0.7288135593220338
1.00297117

In [45]:
total = 0
for data in test_loader:  # Iterate in batches over the training/test dataset.
        
        if data.x.shape[0] == 0:
            continue

        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        
        total += data.y
        
        print(f' {pred} {data.y} {pred == data.y}' )

#         correct += int((pred == data.y).sum())  # Check against ground-truth labels.

 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')

 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')

 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')

 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')

 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
 tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')

 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], devic

 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')
 tensor([0], device='cuda:0') tensor([1], device='cuda:0') tensor([False], device='cuda:0')


In [12]:
total

tensor([174], device='cuda:0')

In [14]:
from torch import nn

In [25]:

loss = nn.CrossEntropyLoss()
inputs = torch.randn(1, 2, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(2)
output = loss(inputs, target)

In [26]:
print(inputs)
print(target)

tensor([[-0.8565, -0.4334]], requires_grad=True)
tensor([1])


In [30]:
loss = nn.CrossEntropyLoss(weight=torch.tensor([1.0,0.0]))
output = loss(inputs, target)
output

tensor(0., grad_fn=<NllLossBackward0>)